## Notebook to prepare the ERA5 data to be able to save the datasets locally

The next step would be to create the datasets for lateral forcing.

In this notebook:
- 

The ERA5 data is:
- 

The data is downloaded from: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels
and from: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-preliminary-back-extension


 

In [2]:
# Import necessary packages

import dask
import xarray as xr

from tqdm import tqdm

dask.config.set({"array.slicing.split_large_chunks": False})

In [3]:
# Load the different datasets as one dataset when they have similar sampling frequency
era5_hourly_tp = xr.open_mfdataset('/Volumes/Iris 300 GB/ERA5/raw/1950_2022/hourly/tp_*.nc')
era5_hourly_ssr = xr.open_mfdataset('/Volumes/Iris 300 GB/ERA5/raw/1950_2022/hourly/ssr_*.nc')

#era5_6hourly = xr.open_mfdataset('/Users/iriskeizer/Documents/ROMS/data/ERA5/raw/1950_2022/6hourly/*.nc')

In [5]:
era5_hourly_ssr.ssr

<xarray.DataArray 'ssr' (time: 622378, latitude: 137, longitude: 153)>
dask.array<concatenate, shape=(622378, 137, 153), dtype=float32, chunksize=(113952, 137, 153), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -24.0 -23.75 -23.5 -23.25 ... 13.5 13.75 14.0
  * latitude   (latitude) float32 66.0 65.75 65.5 65.25 ... 32.5 32.25 32.0
  * time       (time) datetime64[ns] 1950-01-01T07:00:00 ... 2020-12-31T23:00:00
Attributes:
    units:          J m**-2
    long_name:      Surface net solar radiation
    standard_name:  surface_net_downward_shortwave_flux

In [14]:
# Delete 2022
era5_hourly_tp = era5_hourly_tp.where(era5_hourly_tp.time.dt.year < 2022, drop = True)
era5_hourly_ssr = era5_hourly_ssr.where(era5_hourly_ssr.time.dt.year < 2022, drop = True)
#era5_6hourly = era5_6hourly.where(era5_6hourly.time.dt.year < 2022, drop = True)

In [15]:
# Delete attributes as they aren't valid anymore

del era5_hourly_tp.attrs['Conventions']
del era5_hourly_tp.attrs['history']
del era5_hourly_ssr.attrs['Conventions']
del era5_hourly_ssr.attrs['history']
#del era5_6hourly.attrs['Conventions']
#del era5_6hourly.attrs['history']

In [5]:
# Delete expver = 5

#era5_6hourly = era5_6hourly.sel(expver = 1).drop('expver')

In [16]:
# Save data


era5_hourly_tp.tp.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/hourly/era5_hourly_tp_2020.nc')
era5_hourly_ssr.ssr.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/hourly/era5_hourly_ssr_2020.nc')

#era5_6hourly.d2m.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/6hourly/era5_6hourly_d2m.nc')
#era5_6hourly.msl.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/6hourly/era5_6hourly_msl.nc')
#era5_6hourly.t2m.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/6hourly/era5_6hourly_t2m.nc')
#era5_6hourly.tcc.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/6hourly/era5_6hourly_tcc.nc')
#era5_6hourly.v10.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/6hourly/era5_6hourly_v10.nc')
#era5_6hourly.u10.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/6hourly/era5_6hourly_u10.nc')



In [15]:
era5_hourly = xr.merge([era5_hourly_tp, era5_hourly_ssr])

del era5_hourly.attrs['Conventions']
del era5_hourly.attrs['history']

era5_hourly.tp.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/hourly/era5_hourly_tp.nc')
era5_hourly.ssr.to_netcdf('/Users/iriskeizer/Documents/ROMS/data/ERA5/preprocessed/hourly/era5_hourly_ssr.nc')

In [24]:
tp_2013_2019 = xr.open_dataset('/Volumes/Iris 300 GB/ERA5/raw/1950_2022/hourly/tp_2013_2019.nc')

In [31]:
tp_2020 = xr.open_dataset('/Volumes/Iris 300 GB/ERA5/raw/1950_2022/hourly/tp_2020.nc')

In [35]:
tp_2020.sortby('time').time

<xarray.DataArray 'time' (time: 366)>
array(['2020-01-01T00:00:00.000000000', '2020-01-02T00:00:00.000000000',
       '2020-01-03T00:00:00.000000000', ..., '2020-12-29T00:00:00.000000000',
       '2020-12-30T00:00:00.000000000', '2020-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-12-31
Attributes:
    long_name:  time

In [37]:
tp_2013_2019.time

<xarray.DataArray 'time' (time: 61344)>
array(['2013-01-01T00:00:00.000000000', '2013-01-01T01:00:00.000000000',
       '2013-01-01T02:00:00.000000000', ..., '2019-12-31T21:00:00.000000000',
       '2019-12-31T22:00:00.000000000', '2019-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2013-01-01 ... 2019-12-31T23:00:00
Attributes:
    long_name:  time

In [41]:
ssr_2020 = xr.open_dataset('/Volumes/Iris 300 GB/ERA5/raw/1950_2022/hourly/ssr_2020.nc')

In [42]:
ssr_2020.time

<xarray.DataArray 'time' (time: 366)>
array(['2020-01-01T00:00:00.000000000', '2020-01-02T00:00:00.000000000',
       '2020-01-03T00:00:00.000000000', ..., '2020-12-29T00:00:00.000000000',
       '2020-12-30T00:00:00.000000000', '2020-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-12-31
Attributes:
    long_name:  time

In [43]:
d2m_1979_2022 = xr.open_dataset('/Volumes/Iris 300 GB/ERA5/raw/1950_2022/6hourly/d2m_1979_2022.nc')

In [46]:
d2m_1979_2022 = d2m_1979_2022.where(d2m_1979_2022.time.dt.year > 2019, drop = True)

In [47]:
d2m_1979_2022.time

<xarray.DataArray 'time' (time: 3196)>
array(['2020-01-01T00:00:00.000000000', '2020-01-01T06:00:00.000000000',
       '2020-01-01T12:00:00.000000000', ..., '2022-03-09T06:00:00.000000000',
       '2022-03-09T12:00:00.000000000', '2022-03-09T18:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 ... 2022-03-09T18:00:00
Attributes:
    long_name:  time

In [58]:
ssr_2019_2020 = xr.open_dataset('/Volumes/Iris 300 GB/ERA5/raw/1950_2022/hourly/ssr_2019_2020.nc')

In [59]:
ssr_2019_2020 = ssr_2019_2020.where(ssr_2019_2020.time.dt.year > 2019, drop = True)

In [61]:
ssr_2019_2020.nbytes

736557128

In [57]:
ssr_2019_2020.to_netcdf('/Volumes/Iris 300 GB/ERA5/raw/1950_2022/hourly/ssr_2020.nc')